In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

Troubleshooting.

To update Chromedriver download the latest version from 
https://sites.google.com/a/chromium.org/chromedriver/downloads

and put it into /usr/local/bin

In [2]:
SHEET_NAME = 'prepared_sheet'
BROWSER_LINK = 'https://your_link.com/'
EMAIL = 'your_email@email.com'
PASSWORD = 'your_password'
SOURCE_LIB = r'/automation/'
FILE_NAME = r'your_file'
SIMULATION_PAGE = r'https://simulation_page.com/'

In [9]:
new_browser = webdriver.Chrome()

In [3]:
def automate(source_lib,browser,file_name,sheet_name,start_index,end_index):
#     browser.get('BROWSER_LINK')

#     try:
#         element = WebDriverWait(browser, 10).until(
#             EC.presence_of_element_located((By.NAME, "EmailAddress"))
#         )
#     except:
#         print('Cannot load login page')

#     email = browser.find_element_by_name('EmailAddress')
#     email.send_keys(EMAIL)

#     password = browser.find_element_by_name('Password')
#     password.send_keys(PASSWORD + Keys.RETURN)

#     time.sleep(8)
    
    alpha_xl = pd.ExcelFile(source_lib + file_name)
    dataset = sheet_name
    alpha_df = alpha_xl.parse(dataset)
    
    df = alpha_df.copy(deep=True).loc[0:len(alpha_df)]
    
    index = start_index
    print('start index is', index)
    if (end_index==-1) or (end_index>=len(df)):
        end_index = len(df)
    while index < end_index:
        alpha = df.loc[index, 'alpha']
        
        print('loading simultation page')
        
        # Load simulate page
        browser.get(SIMULATION_PAGE)

        # Wait until insertion of the code is loaded
        try:
            element = WebDriverWait(browser, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "CodeMirror-placeholder"))
            )
        except:
            print('Cannot load codeline')
        finally:
            # Insert the alpha code into the box
            code_form = browser.find_element_by_class_name('CodeMirror-placeholder')
            actions = ActionChains(browser)
            actions.click(code_form)
            alpha_code = alpha + '                 '
            actions.send_keys(alpha_code)
            actions.perform()

            # Click the simulate button
            browser.find_element_by_css_selector('.btn.btn-primary.btn-sm.sim-action-simulate').click()

            # Wait until simulation completed
            try:
                element = WebDriverWait(browser, 400).until( # was 600
                    EC.presence_of_element_located((By.CLASS_NAME, "highcharts-container"))
                )
            except:
                print('Simulation too long')
            finally:
                time.sleep(2)
                browser.save_screenshot(source_lib + r'screenshots/' + dataset + str(index) + r'PnL.png')
                # Click stats tab and take a screenshot
                stats = browser.find_element_by_id('test-statsBtn').click()
                time.sleep(2)
                browser.save_screenshot(source_lib + r'screenshots/' + dataset + str(index) + r'stat.png')

                index += 1

In [ ]:
automate(SOURCE_LIB,new_browser,FILE_NAME,SHEET_NAME,0,-1)

## Close the browser window

In [ ]:
browser.quit()